<a href="https://colab.research.google.com/github/hyeonsang010716/Recommendation-system/blob/main/user_free_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!unzip hetrec2011-lastfm-2k.zip

--2024-04-21 12:58:05--  https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589075 (2.5M) [application/zip]
Saving to: ‘hetrec2011-lastfm-2k.zip’

hetrec2011-lastfm-2 100%[===================>]   2.47M  --.-KB/s    in 0.1s    

2024-04-21 12:58:05 (17.7 MB/s) - ‘hetrec2011-lastfm-2k.zip’ saved [2589075/2589075]

Archive:  hetrec2011-lastfm-2k.zip
  inflating: user_friends.dat        
  inflating: user_taggedartists.dat  
  inflating: user_taggedartists-timestamps.dat  
  inflating: artists.dat             
  inflating: readme.txt              
  inflating: tags.dat                
  inflating: user_artists.dat        


In [ ]:
!head artists.dat

head: cannot open 'artists.dat' for reading: No such file or directory


In [ ]:
artists = {}
with open("artists.dat" , "r") as f:
  f.readline()
  for line in f:
    id , name , _ , _ = line.split("\t")
    artists[int(id)] = name

In [ ]:
import numpy as np
users = []
items = []
weights = []
with open("user_artists.dat" , "r") as f:
  f.readline()
  for line in f:
    uid , aid , weight = line.split("\t")
    users.append(int(uid))
    items.append(int(aid))
    weights.append(np.log(1 + float(weight)))
users = np.array(users)
items = np.array(items)
weights = np.array(weights)

In [ ]:
from scipy.sparse import csc_matrix
from sklearn.linear_model import ElasticNet
from tqdm import tqdm
rows = []
cols = []
data = []
# csc는 (rating  , (row , col))
train = csc_matrix((weights , (users , items)))
model = ElasticNet(alpha = 0.1 , l1_ratio = 0.5, positive = True , fit_intercept = False, copy_X = False , selection = 'random' , tol = 1e-6 , max_iter = 100)
n_items = max(items) + 1
for current_item in tqdm(range(n_items)):
  y = train[: , current_item].toarray()
  if all(y == 0): continue
  s = train.indptr[current_item]
  e = train.indptr[current_item + 1]
  backup = train.data[s:e].copy()
  train.data[s:e] = 0.0
  model.fit(train , y)
  scoef = model.sparse_coef_
  cols.extend([current_item] * scoef.indices.shape[0])
  rows.extend(scoef.indices)
  data.extend(scoef.data)

  train.data[s:e] = backup
W = csc_matrix( (data , (rows , cols)) , shape = (n_items , n_items))


100%|██████████| 18746/18746 [03:26<00:00, 90.67it/s] 


In [ ]:
# 1918 동방신기
q = 1918
s = W.indptr[q]
e = W.indptr[q+1]
artist_ids = W.indices[s:e]
scores = W.data[s:e]
print("[{}] 와 유사한 artists:".format(artists[q]))
for s , aid in sorted(zip(scores , artist_ids) , reverse = True):
  print(s , aid , artists[aid])

[동방신기] 와 유사한 artists:
0.10777798273670769 2092 소녀시대
0.09632058048178652 393 Super Junior
0.023374301779661585 2094 BoA
0.014559506414026214 2097 Brown Eyed Girls
0.007568961329812519 2799 f(x)
0.0030389594715197385 55 Kylie Minogue
0.0010564176005322219 292 Christina Aguilera
0.0007849191333476713 2102 倖田來未


In [ ]:
uid = 312
userdata = train[uid , :].tocsr()

scores = userdata.dot(W)
for s, aid in sorted(zip(scores.data , scores.indices) , reverse = True)[:100]:
  if aid not in userdata.indices:
    print(s , aid , artists[aid])


3.6422400417739698 374 宇多田ヒカル
2.969105007171338 1904 SHINee
2.113919847040355 2091 4minute
1.7813714261171147 2101 Capsule
1.7016521388170187 289 Britney Spears
1.5240928866510346 464 3OH!3
1.3433195302773033 400 2NE1
1.196476883950902 2089 티아라
1.0337634311579396 288 Rihanna
0.998296598898178 300 Katy Perry
0.9934655071104239 1857 YUI
0.9886975236597993 409 G-Dragon
0.9757354339775685 377 Linkin Park
0.9549264629252869 306 Black Eyed Peas
0.9111230917250542 691 Owl City
0.8356358967326332 257 Mariah Carey
0.8309113317545049 2104 ルルティア
0.6877432882888026 1919 HIGH and MIGHTY COLOR
0.5804573359670403 1246 Panic! At the Disco
0.5665611209888506 906 Girls Aloud
0.5135718915402959 379 中島美嘉
0.4980561518183765 55 Kylie Minogue
0.4741839322973734 456 Jesse McCartney
0.428856821792848 689 Florence + the Machine
0.41904625903766785 344 Taylor Swift
0.40201089159062275 3187 Kudai
0.3977596155851986 3490 Utada
0.38165724118865485 1243 Mika
0.37814479146007635 1043 Boys Like Girls
0.373728548801798

In [ ]:
for x in userdata.indices:
  print(artists[x])

Lady Gaga
Cobra Starship
BIG BANG
2PM
浜崎あゆみ
B2ST
Super Junior
山下智久
NEWS
嵐
After School
Glee Cast
東方神起
동방신기
Perfume
Alan
소녀시대
BoA
Brown Eyed Girls
Kara
倖田來未
U-KISS
f(x)
이효리
黒木メイサ
Belanova
安室奈美恵
The Lonely Island
Wonder Girls
KAT-TUN
AAA
モーニング娘。
大塚愛
GIRL NEXT DOOR
샤이니
V6
後藤真希
AKB48
Clazziquai Project
신화
w-inds.
SE7EN
Sweetbox
Banana Girl
KinKi Kids
林俊傑
TOKIO
蔡依林
SS501
Sweet Vacation
